In [1]:
%pip install keras-cv
%pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 10.2 MB/s eta 0:00:00


In [2]:
%pip install tf-models-official
%pip install tensorflow-object-detection-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 31.5 MB/s eta 0:0

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert class vectors to binary class matrices
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Split the training data into training and validation sets
val_images = train_images[:5000]
val_labels = train_labels[:5000]
train_images = train_images[5000:]
train_labels = train_labels[5000:]

# Create ImageDataGenerator for resizing
datagen = ImageDataGenerator(
    # Add any other preprocessing here
    rescale=1./255
)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


## **Inference with a pretrained classifier:**

In [4]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Resizing

# Load MobileNetV2 with ImageNet weights as a base model
# Adjust the input_shape to the new image size
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Create a new model on top
model = Sequential([
    Resizing(32, 32),  # Resizing to 128x128
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(10, activation='softmax')  # CIFAR-10 has 10 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Prepare the ImageDataGenerator with the new size
datagen = ImageDataGenerator(
    rescale=1./255
)

# Prepare the data generators
train_gen = datagen.flow(train_images, train_labels, batch_size=32)
val_gen = datagen.flow(val_images, val_labels, batch_size=32)
test_gen = datagen.flow(test_images, test_labels, batch_size=32)

# Train the model
model.fit(train_gen, epochs=2, validation_data=val_gen)

# Evaluate the model
loss, accuracy = model.evaluate(test_gen)
print("Test Accuracy:", accuracy)

<ipython-input-4-76ba9c807dff>:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1407/1407 ━━━━━━━━━━━━━━━━━━━━ 359s 232ms/step - accuracy: 0.3096 - loss: 2.0067 - val_accuracy: 0.1040 - val_loss: 3.0933
Epoch 2/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 378s 229ms/step - accuracy: 0.5266 - loss: 1.3561 - val_accuracy: 0.1040 - val_loss: 4.8139
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.1056 - loss: 4.8136
Test Accuracy: 0.10010000318288803


## **Fine-tuning a pretrained backbone:**

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert class vectors to binary class matrices
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Split the training data into training and validation sets
val_images = train_images[:5000]
val_labels = train_labels[:5000]
train_images = train_images[5000:]
train_labels = train_labels[5000:]

# Create ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_gen = datagen.flow(train_images, train_labels, batch_size=32)
val_gen = datagen.flow(val_images, val_labels, batch_size=32)
test_gen = datagen.flow(test_images, test_labels, batch_size=32)

# Load MobileNetV2 with ImageNet weights as a base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model
base_model.trainable = False

# Create a new model on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(10, activation='softmax')  # CIFAR-10 has 10 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the top layers
model.fit(train_gen, epochs=2, validation_data=val_gen)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
fine_tune_at = 100  # This is an example, adjust based on model architecture

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

# Compile the model with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model
model.fit(train_gen, epochs=2, validation_data=val_gen)

# Evaluate the model
loss, accuracy = model.evaluate(test_gen)
print("Test Accuracy:", accuracy)

<ipython-input-5-26b03ac766c8>:34: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


Epoch 1/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 91s 61ms/step - accuracy: 0.1149 - loss: 2.2961 - val_accuracy: 0.1584 - val_loss: 2.2436
Epoch 2/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 87s 61ms/step - accuracy: 0.1534 - loss: 2.2378 - val_accuracy: 0.1532 - val_loss: 2.2065
Epoch 1/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 220s 146ms/step - accuracy: 0.1325 - loss: 3.7850 - val_accuracy: 0.1172 - val_loss: 2.7876
Epoch 2/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 259s 144ms/step - accuracy: 0.1909 - loss: 2.3221 - val_accuracy: 0.0918 - val_loss: 2.3540
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.0983 - loss: 2.3571
Test Accuracy: 0.09300000220537186


## **Training a image classifier from scratch:**

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert class vectors to binary class matrices
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Split the training data into training and validation sets
val_images = train_images[:5000]
val_labels = train_labels[:5000]
train_images = train_images[5000:]
train_labels = train_labels[5000:]

# Create ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Create a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels), batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print("Test Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 86s 119ms/step - accuracy: 0.4314 - loss: 1.5865 - val_accuracy: 0.5748 - val_loss: 1.2263
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 82s 117ms/step - accuracy: 0.6442 - loss: 1.0041 - val_accuracy: 0.5938 - val_loss: 1.1898
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 141s 117ms/step - accuracy: 0.7034 - loss: 0.8304 - val_accuracy: 0.6664 - val_loss: 0.9707
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 83s 118ms/step - accuracy: 0.7512 - loss: 0.7096 - val_accuracy: 0.6750 - val_loss: 0.9463
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 141s 117ms/step - accuracy: 0.7810 - loss: 0.6197 - val_accuracy: 0.6608 - val_loss: 1.0219
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 142s 117ms/step - accuracy: 0.8021 - loss: 0.5622 - val_accuracy: 0.6818 - val_loss: 0.9588
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 142s 118ms/step - accuracy: 0.8290 - loss: 0.4886 - val_accuracy: 0.6956 - val_loss: 0.9508
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 141s 117ms/step - accuracy: 0.8462 - lo